In [3]:
# 1) Design model (input size, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training loop
#    - forward pass: compute prediction and loss
#    - backward pass: gradients
#    - update weights

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
# 0) Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)
print(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


569 30
[0 1]
torch.Size([455, 30]) torch.Size([455])
torch.Size([114, 30]) torch.Size([114])
torch.Size([455, 30]) torch.Size([455, 1])
torch.Size([114, 30]) torch.Size([114, 1])


In [13]:
# model: y = w0x0 + w1x1 + b, sigmoid at the end
input_size = n_features
output_size = 1 # binary

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return torch.sigmoid(self.lin(x))

model = LinearRegression(input_size, output_size)

#loss criterion and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [16]:
# Training loop
n_iters = 200

for epoch in range(n_iters):
    # forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # backward pass
    loss.backward()

    # update weights
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch {epoch+1}: loss = {loss: 0.8f}')

# Evaluation
with torch.no_grad():
    y_eva = model(X_test)
    y_eva_cls = y_eva.round()
    acc = y_eva_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'acc = {acc:.4f}')

epoch 10: loss =  0.17208649
epoch 20: loss =  0.16849808
epoch 30: loss =  0.16514289
epoch 40: loss =  0.16199709
epoch 50: loss =  0.15904003
epoch 60: loss =  0.15625384
epoch 70: loss =  0.15362300
epoch 80: loss =  0.15113385
epoch 90: loss =  0.14877442
epoch 100: loss =  0.14653404
epoch 110: loss =  0.14440328
epoch 120: loss =  0.14237373
epoch 130: loss =  0.14043783
epoch 140: loss =  0.13858879
epoch 150: loss =  0.13682044
epoch 160: loss =  0.13512729
epoch 170: loss =  0.13350432
epoch 180: loss =  0.13194691
epoch 190: loss =  0.13045090
epoch 200: loss =  0.12901251
acc = 0.9298
